
### Satellite Imagery Querying, Parsing and Rendering

In [6]:
import bqplot
import datetime
import dateutil.parser
import ee
import ipywidgets
import ipyleaflet
import IPython.display
import numpy as np
import pprint
import pandas as pd
import traitlets
from humanize import naturalsize as size


In [3]:
pp = pprint.PrettyPrinter(depth=4)


In [59]:
# initialize google earth engine
ee.Initialize()

#### Filtering collections by time period and cloud cover

In [8]:
# public image collections
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
        .filterDate('2018-08-01', '2018-12-01')\
        .filterMetadata('CLOUD_COVER', 'less_than', 2)

print(f'Total number of assets with filters: {l8sr.size().getInfo()}')
print(f'Total size of LS-8 collection: {size(l8sr.reduceColumns(ee.Reducer.sum(), ["system:asset_size"]).getInfo()["sum"])}')


Total number of assets with filters: 10681
Total size of LS-8 collection: 6.2 TB


#### Get image from a collection and query its metadata

In [14]:
sample_image = ee.Image(l8sr.first())
pp.pprint(sample_image.getInfo())
band_names_original = sample_image.bandNames()
band_names_original.getInfo()

{'bands': [{'crs': 'EPSG:32628',
            'crs_transform': [30, 0, 341385, 0, -30, 8807715],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [9161, 9161],
            'id': 'B1'},
           {'crs': 'EPSG:32628',
            'crs_transform': [30, 0, 341385, 0, -30, 8807715],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [9161, 9161],
            'id': 'B2'},
           {'crs': 'EPSG:32628',
            'crs_transform': [30, 0, 341385, 0, -30, 8807715],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [9161, 9161],
            'id':

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B10',
 'B11',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

In [63]:
def get_tile_layer_url(ee_image_object):
    """
    Returns a tile layer url from the earthengine server
    """
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'
    return tile_url_template.format(**map_id)

In [77]:
# create a slider widget to add the Landsat 8 satellite imagery
map1 = ipyleaflet.Map(
    center=(37.4112, -122.0634),
    zoom=12,
    layout={'height': '500px'},
)

l8sr_false_color = get_tile_layer_url(l8sr.median().visualize(min=100, max=3500, gamma=1.5, bands=['B4', 'B3', 'B2']))
l8sr_true_color = get_tile_layer_url(l8sr.median().visualize(min=100, max=3500, gamma=1.5, bands=['B5', 'B3', 'B2']))
left = ipyleaflet.TileLayer(url=l8sr_false_color)
right = ipyleaflet.TileLayer(url=l8sr_true_color)
map1.add_layer(right)
map1.add_layer(left)
control = ipyleaflet.SplitMapControl(left_layer=left, right_layer=right)
map1.add_control(control)
map1

Map(center=[37.4112, -122.0634], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

In [78]:
map1.layers

(TileLayer(attribution='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors', base=True, max_zoom=19, min_zoom=1, name='OpenStreetMap.Mapnik', options=['attribution', 'bounds', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms', 'zoom_offset'], url='https://tile.openstreetmap.org/{z}/{x}/{y}.png'),
 TileLayer(options=['attribution', 'bounds', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms', 'zoom_offset'], url='https://earthengine.googleapis.com/map/projects/earthengine-legacy/maps/d659877987fe7e63701ab10eb305c81e-9406a39a6124094823bd4a1aea2226af/{z}/{x}/{y}?token='),
 TileLayer(options=['attribution', 'bounds', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms', 'zoom_offset'], url='https://earthengine.googleapis.com/map/projects/earthengine-legacy/maps/8cff9ac667f68338ae0181

#### Generating an Index

In [65]:
# NDVI

img = l8sr.median()
red = img.select('B4').rename('red')
nir = img.select('B5').rename('nir')

# ndvi = (nir.subtract(red).divide(nir.add(red))).rename('ndvi')
ndvi = img.normalizedDifference(['B5', 'B4']).rename('ndvi')

In [67]:
map2 = ipyleaflet.Map(
    center=(37.4112, -122.0634),
    zoom=12,
    layout={'height': '500px'},
)

ndvi_tile_url = get_tile_layer_url(ndvi.visualize(min=-0.2, max=0.6, palette='grey,yellow,green'))
layer = ipyleaflet.TileLayer(url=ndvi_tile_url)
map2.add_layer(ndvi)
map2

Map(center=[37.4112, -122.0634], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

AttributeError: type object 'Map' has no attribute 'centerObject'